## Una introducción práctica a Pytorch, https://pytorch.org/

El objetivo de esta primera lección es el de arrojar un vistazo preliminar a las capacidades de la librería pytorch. Pytorch es un framework de deep learning con un estilo muy familiar al de numpy (https://numpy.org/).

Como recordatorio, los objetos principales de numpy (y en cualquier librería de cálculo científico) son los arrays, y las operaciones que podemos calcular sobre ellos, por ejemplo:

In [1]:
# Para instalar las últimas versiones
# !pip install torch torchvision

In [1]:
import numpy as np

A = np.array([[1., 2.], [3., 4.]])
B = np.array([[0., 1.], [0., 1.]])

In [2]:
C = A + B
C

array([[1., 3.],
       [3., 5.]])

In [3]:
np.sum(A)

10.0

Pytorch es similar, solo que los objetos principales reciben el nombre de tensores (arrays multidimensionales)

In [4]:
import torch

A = torch.tensor([[1., 2.], [3., 4.]])
B = torch.tensor([[0., 1.], [0., 1.]])

In [5]:
C = A + B
C

tensor([[1., 3.],
        [3., 5.]])

In [6]:
torch.sum(A)

tensor(10.)

¡Así de simple es! Casi cualquier función de numpy (y de scipy) tiene un equivalente en pytorch. La lista completa de funciones puede consultarse en https://pytorch.org/docs/stable/torch.html

Podemos convertir entre arrays de numpy y tensores de torch usando

In [7]:
C.numpy()

array([[1., 3.],
       [3., 5.]], dtype=float32)

In [8]:
torch.from_numpy(C.numpy())

tensor([[1., 3.],
        [3., 5.]])

Un atributo de los tensores muy útil (especialmente para depurar código) es .shape, que nos devuelve las dimensiones de nuestro tensor:

In [9]:
C.shape

torch.Size([2, 2])

### Entonces, ¿por qué usar pytorch en vez de numpy?

Hasta ahora, parece que pytorch hace lo mismo que numpy. Pero tiene un montón de extensiones que lo hacen especialmente útil para aplicaciones de ML/AI. Veamos las más importantes

1. **Autograd** (diferenciación automática)

2. **GPU**

3. **Abstracciones** para ML

#### Diferenciación automática

Pytorch puede calcular el gradiente de cualquier función que podáis escribir utilizando tensores. No necesitamos calcular las derivadas (gradientes) a mano. Pytorch tampoco las calcula mediante diferencias numéricas (daría lugar a resultados aproximados o peor aún, inestables). En su lugar, pytorch va llevando un registro de todas las operaciones que definimos, y luego aplica la regla de la cadena estratégicamente (de forma simbólica), con lo que el resultado es una derivada exacta: https://en.wikipedia.org/wiki/Automatic_differentiation

Para hacer esto, solo tenemos que activar un flag en las variables sobre las cuales queramos derivar/calcular gradiente. Por ejemplo, definamos una función $f(x) = \sum_{i=1}^{10} x_i^2$, donde $x \in \mathbb{R}^{10}$, y supongamos que queramos calcular $\nabla f(x)$ en $x = (1, 1, \ldots, 1)$

Primero, definimos el input. Esta vez, en vez de usar torch.tensor, usaremos torch.ones, similar a numpy.
Pero fijémonos en que hemos añadido la opción de que estamos interesados en calcular gradientes respecto a esta variable en el futuro próximo.

In [10]:
x = torch.ones(10, requires_grad=True)
x

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], requires_grad=True)

Ahora, definimos las operaciones de la función. Podríamos meter todo dentro de una función de python, pero esto no es necesario realmente:

In [11]:
y = torch.sum(x**2)
y

tensor(10., grad_fn=<SumBackward0>)

Y finalmente, solo tenemos que invocar al método backward() para calcular la derivada $\frac{\partial y}{\partial x}$. Entonces, podemos consular el atributo .grad de cualquier variables de input (para la que hayamos activado el flag requires_grad)

In [12]:
y.backward()

In [13]:
x.grad

tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

Esto era un ejemplo muy sencillo, pero lo cierto es que se pueden calcular derivadas a través de código muy complejo:

In [14]:
x = torch.ones(5, requires_grad=True)

In [15]:
z = x
while z[0] >= 0.2:
    z = torch.sin(z)
y = torch.sum(z)

In [16]:
y.backward()
x.grad

tensor([0.0062, 0.0062, 0.0062, 0.0062, 0.0062])

**Ejercicio** Dada $f(x_1,x_2) = sen(x_1)*cos(x_1*x_2)^2$, calcula $\frac{\partial f}{\partial x_1}$ en $(x_1, x_2) = (2, 2)$

#### Aceleración por tarjeta gráfica (GPU)

La CPU de vuestros ordenadores posiblemente tendrá entre 4 y 16 cores, así que la paralelización es algo limitada. Si tenéis alguna tarjeta gráfica de nvidia con los drivers de cuda instalados, podéis usarla para acelerar cálculos sobre arrays (pues la GPU tiene muchos más cores)

Veamos un ejemplo sencillo de calcular el cuadrado de una matriz aleatoria grande, utilizando numpy y pytorch en CPU, y pytorch en GPU

In [22]:
A = np.random.randn(5000, 5000)

In [23]:
%%timeit
B = A ** 2

28.6 ms ± 659 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
A = torch.from_numpy(A)

In [25]:
%%timeit
B = A ** 2

12.3 ms ± 249 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Con pytorch, podemos calcular sobre la GPU simplemente invocando .to('cuda')

In [26]:
A = A.to('cuda')

In [27]:
%%timeit
B = A ** 2

2.46 ms ± 53.4 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Impresionante! Hemos conseguido reducir el tiempo de cálculo de 28 ms a 2.5 ms, 10 veces más rápido!!

#### Abstracciones para deep learning

Además de lo anterior, pytorch nos ofrece sublibrerías conteniendo funciones ya definidas para deep learning.
Por ejemplo:

1. torch.nn contiene varios tipos de capas para las redes neuronales (https://pytorch.org/docs/stable/nn.html)

2. torch.optim contiene varios optimizadores implementados, como SGD, Adam, etc (https://pytorch.org/docs/stable/optim.html)

Cubriremos todo esto en los próximos cuadernos!